# Telecom X - Modelado de Evasión de Clientes

Este cuaderno carga los datos tratados de Telecom X, realiza el preprocesamiento necesario para preparar los datos y entrena modelos de **Regresión Logística** y **Random Forest** para predecir el **churn** de clientes. Al final se evalúan los modelos y se visualiza la importancia de las variables.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Cargar el dataset tratado
df_encoded = pd.read_csv('TelecomX_Data_Treated.csv')
print('Shape:', df_encoded.shape)
df_encoded.head()

In [ ]:
# Separar variables (X) y objetivo (y)
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Escalado para regresión logística
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar Regresión Logística
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

# Entrenar Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Evaluar
results = {}
results['Logistic Regression'] = {
    'Accuracy': accuracy_score(y_test, y_pred_lr),
    'Precision': precision_score(y_test, y_pred_lr),
    'Recall': recall_score(y_test, y_pred_lr),
    'F1-score': f1_score(y_test, y_pred_lr),
    'Confusion Matrix': confusion_matrix(y_test, y_pred_lr)
}
results['Random Forest'] = {
    'Accuracy': accuracy_score(y_test, y_pred_rf),
    'Precision': precision_score(y_test, y_pred_rf),
    'Recall': recall_score(y_test, y_pred_rf),
    'F1-score': f1_score(y_test, y_pred_rf),
    'Confusion Matrix': confusion_matrix(y_test, y_pred_rf)
}

pd.DataFrame(results)

In [ ]:
# Matrices de confusión y gráficos de importancia de variables
for modelo, metricas in results.items():
    cm = metricas['Confusion Matrix']
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de confusión - {modelo}')
    plt.xlabel('Predicho')
    plt.ylabel('Real')
    plt.show()

# Importancias Random Forest
import pandas as pd
fi = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(10,5))
fi.head(15).plot(kind='bar')
plt.title('Top 15 Importancias - Random Forest')
plt.ylabel('Importancia')
plt.show()

# Importancias regresión logística
coef = pd.Series(lr.coef_[0], index=X.columns).abs().sort_values(ascending=False)
plt.figure(figsize=(10,5))
coef.head(15).plot(kind='bar')
plt.title('Top 15 Importancias - Regresión Logística')
plt.ylabel('Magnitud del coeficiente')
plt.show()

In [ ]:
# Matriz de correlación y boxplots
plt.figure(figsize=(12,8))
sns.heatmap(df_encoded.corr(), cmap='coolwarm', center=0)
plt.title('Matriz de correlación - Datos codificados')
plt.show()

plt.figure(figsize=(6,4))
sns.boxplot(x='Churn', y='tenure', data=df_encoded)
plt.title('Tenure vs Churn')
plt.show()

plt.figure(figsize=(6,4))
sns.boxplot(x='Churn', y='TotalCharges', data=df_encoded)
plt.title('Total Charges vs Churn')
plt.show()

## Conclusiones

El modelo de regresión logística obtuvo un mejor equilibrio entre precisión y recall que el Random Forest. Las variables de mayor relevancia incluyen **tenure**, **TotalCharges**, **MonthlyCharges**, el uso de **InternetService\_Fiber optic** y el método de pago **Electronic check**. Clientes con mayor antigüedad y contratos de larga duración presentan menor probabilidad de cancelar, mientras que cuotas mensuales altas y determinados métodos de pago incrementan el riesgo de churn.
